In [45]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import json
from PIL import Image
from nibabel.affines import apply_affine
import numpy.linalg as npl
import os
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer
import torch
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt
import cv2
import pandas as pd
import torchvision.transforms as T
from nibabel.affines import apply_affine
import numpy.linalg as npl
import copy
import cv 
import random
from detectron2.structures import BoxMode
import torchio as tio

In [17]:
data_path = Path("../frcnn_data/synthetic")
dataset_size = 1000
high_noise = 50
low_noise = 10

In [14]:
#if not data_path.exists():
#    data_path.mkdir(parents=True, exist_ok=True)
#    (data_path / "train").mkdir(parents=True, exist_ok=True)
#    (data_path / "test").mkdir(parents=True, exist_ok=True)

#for file in chain(data_path.glob("*/*")):
#    os.remove(file)

In [18]:
def draw_brain(img, noise):
    brain_center = np.random.randint(150,400,2)
    brain_size = np.random.randint(70, 100)

    brain = np.zeros(img.shape, np.uint8) # (img_h, img_w, channels)
    cv.circle(brain, brain_center, brain_size, (128,128,128), -1)
    brain = np.where(brain > 0, np.random.normal(brain, noise).astype(int), 0)

    mask = brain > 0
    img[mask] = brain[mask]

    return np.append(brain_center, brain_size) # (x,y,r)

def draw_tumor(img, brain, noise):
    # find center within brain
    ang = random.uniform(0, 1) * 2 * pi
    hyp = sqrt(random.uniform(0, 1)) * (brain[2] - 10)
    tumor_center = [int(brain[0]+(cos(ang)) * hyp), int(brain[1]+(sin(ang) * hyp))]

    # find size within brain
    max_dist = brain[2] - sqrt((brain[0] - tumor_center[0])**2 + (brain[1] - tumor_center[1])**2)
    tumor_size = np.random.randint(5, min(max_dist, 16))

    tumor = np.zeros(img.shape, np.uint8)
    cv.circle(tumor, tumor_center, tumor_size, (0,255,0), -1)
    tumor = np.where(tumor > 0, np.random.normal(tumor, noise).astype(int), 0)

    mask = tumor > 0
    img[mask] = tumor[mask]

    return np.append(tumor_center, tumor_size) # (x,y,r)

def make_entry(path, pt, positive, noise, img_h=512, img_w=512, channels=3):
    pt = f"Cmet{pt:04}"
    print(pt)
    offset = np.random.randint(-50,50)
    img_h += offset
    img_w += offset

    img = np.zeros((img_h, img_w, channels),np.uint8)
    brain_info = draw_brain(img, noise=noise)

    #subdir = dir

    annotations = []

    if (np.random.randint(10) < (10*positive)):
        #subdir = dir / "anonymized"
        
        tumor_info = draw_tumor(img, brain_info, noise=noise)
        tumor_info[0] -= tumor_info[2]
        tumor_info[1] -= tumor_info[2]
        tumor_info[2] *= 2

        annotations.append(
            {
                "bbox": np.append(tumor_info, tumor_info[-1]).tolist(),
                "bbox_mode": BoxMode.XYWH_ABS,
                "category_id": 0
            }
        )

        # rtss_folder = dir / "rtss"
        # if not rtss_folder.exists():
        #     rtss_folder.mkdir(parents=True, exist_ok=True)

        # np.save(rtss_folder / f"Cmet{pt:04}", tumor_info)
    # else:
    #     subdir = dir / "controls"
    
    data = {
        "file_name": f"{pt}.npy",
        "image_id": str(pt),
        "height": img_h,
        "width": img_w,
        "annotations": annotations
    }
    
    np.save(path/pt, img)

    return data

def create_dataset(size, noise_level, noise, test_split=0.3, positive=0.5):
    train_split = 1.0 - test_split
    assert(train_split > 0)
    global data_path
    train_records = []
    test_records = []

    for num in range(size):
        if np.random.randint(100) < (float(100)*test_split):
            test_records.append(make_entry(data_path / f"{noise_level}/test", num, positive, noise))
        else:
            train_records.append(make_entry(data_path / f"{noise_level}/train", num, positive, noise))
    
    with open(data_path /noise_level /"train/dataset.json", "w") as json_file:
        json_data = json.dumps(train_records, indent=4)
        json_file.write(json_data)

    with open(data_path / noise_level/"test/dataset.json", "w") as json_file:
        json_data = json.dumps(test_records, indent=4)
        json_file.write(json_data)

        
create_dataset(dataset_size, "low_noise", low_noise)
create_dataset(dataset_size, "high_noise", high_noise)

Cmet0000
Cmet0001
Cmet0002
Cmet0003
Cmet0004
Cmet0005
Cmet0006
Cmet0007
Cmet0008
Cmet0009
Cmet0010
Cmet0011
Cmet0012
Cmet0013
Cmet0014
Cmet0015
Cmet0016
Cmet0017
Cmet0018
Cmet0019
Cmet0020
Cmet0021
Cmet0022
Cmet0023
Cmet0024
Cmet0025
Cmet0026
Cmet0027
Cmet0028
Cmet0029
Cmet0030
Cmet0031
Cmet0032
Cmet0033
Cmet0034
Cmet0035
Cmet0036
Cmet0037
Cmet0038
Cmet0039
Cmet0040
Cmet0041
Cmet0042
Cmet0043
Cmet0044
Cmet0045
Cmet0046
Cmet0047
Cmet0048
Cmet0049
Cmet0050
Cmet0051
Cmet0052
Cmet0053
Cmet0054
Cmet0055
Cmet0056
Cmet0057
Cmet0058
Cmet0059
Cmet0060
Cmet0061
Cmet0062
Cmet0063
Cmet0064
Cmet0065
Cmet0066
Cmet0067
Cmet0068
Cmet0069
Cmet0070
Cmet0071
Cmet0072
Cmet0073
Cmet0074
Cmet0075
Cmet0076
Cmet0077
Cmet0078
Cmet0079
Cmet0080
Cmet0081
Cmet0082
Cmet0083
Cmet0084
Cmet0085
Cmet0086
Cmet0087
Cmet0088
Cmet0089
Cmet0090
Cmet0091
Cmet0092
Cmet0093
Cmet0094
Cmet0095
Cmet0096
Cmet0097
Cmet0098
Cmet0099
Cmet0100
Cmet0101
Cmet0102
Cmet0103
Cmet0104
Cmet0105
Cmet0106
Cmet0107
Cmet0108
Cmet0109
Cmet0110
C

Cmet0913
Cmet0914
Cmet0915
Cmet0916
Cmet0917
Cmet0918
Cmet0919
Cmet0920
Cmet0921
Cmet0922
Cmet0923
Cmet0924
Cmet0925
Cmet0926
Cmet0927
Cmet0928
Cmet0929
Cmet0930
Cmet0931
Cmet0932
Cmet0933
Cmet0934
Cmet0935
Cmet0936
Cmet0937
Cmet0938
Cmet0939
Cmet0940
Cmet0941
Cmet0942
Cmet0943
Cmet0944
Cmet0945
Cmet0946
Cmet0947
Cmet0948
Cmet0949
Cmet0950
Cmet0951
Cmet0952
Cmet0953
Cmet0954
Cmet0955
Cmet0956
Cmet0957
Cmet0958
Cmet0959
Cmet0960
Cmet0961
Cmet0962
Cmet0963
Cmet0964
Cmet0965
Cmet0966
Cmet0967
Cmet0968
Cmet0969
Cmet0970
Cmet0971
Cmet0972
Cmet0973
Cmet0974
Cmet0975
Cmet0976
Cmet0977
Cmet0978
Cmet0979
Cmet0980
Cmet0981
Cmet0982
Cmet0983
Cmet0984
Cmet0985
Cmet0986
Cmet0987
Cmet0988
Cmet0989
Cmet0990
Cmet0991
Cmet0992
Cmet0993
Cmet0994
Cmet0995
Cmet0996
Cmet0997
Cmet0998
Cmet0999
Cmet0000
Cmet0001
Cmet0002
Cmet0003
Cmet0004
Cmet0005
Cmet0006
Cmet0007
Cmet0008
Cmet0009
Cmet0010
Cmet0011
Cmet0012
Cmet0013
Cmet0014
Cmet0015
Cmet0016
Cmet0017
Cmet0018
Cmet0019
Cmet0020
Cmet0021
Cmet0022
Cmet0023
C

Cmet0824
Cmet0825
Cmet0826
Cmet0827
Cmet0828
Cmet0829
Cmet0830
Cmet0831
Cmet0832
Cmet0833
Cmet0834
Cmet0835
Cmet0836
Cmet0837
Cmet0838
Cmet0839
Cmet0840
Cmet0841
Cmet0842
Cmet0843
Cmet0844
Cmet0845
Cmet0846
Cmet0847
Cmet0848
Cmet0849
Cmet0850
Cmet0851
Cmet0852
Cmet0853
Cmet0854
Cmet0855
Cmet0856
Cmet0857
Cmet0858
Cmet0859
Cmet0860
Cmet0861
Cmet0862
Cmet0863
Cmet0864
Cmet0865
Cmet0866
Cmet0867
Cmet0868
Cmet0869
Cmet0870
Cmet0871
Cmet0872
Cmet0873
Cmet0874
Cmet0875
Cmet0876
Cmet0877
Cmet0878
Cmet0879
Cmet0880
Cmet0881
Cmet0882
Cmet0883
Cmet0884
Cmet0885
Cmet0886
Cmet0887
Cmet0888
Cmet0889
Cmet0890
Cmet0891
Cmet0892
Cmet0893
Cmet0894
Cmet0895
Cmet0896
Cmet0897
Cmet0898
Cmet0899
Cmet0900
Cmet0901
Cmet0902
Cmet0903
Cmet0904
Cmet0905
Cmet0906
Cmet0907
Cmet0908
Cmet0909
Cmet0910
Cmet0911
Cmet0912
Cmet0913
Cmet0914
Cmet0915
Cmet0916
Cmet0917
Cmet0918
Cmet0919
Cmet0920
Cmet0921
Cmet0922
Cmet0923
Cmet0924
Cmet0925
Cmet0926
Cmet0927
Cmet0928
Cmet0929
Cmet0930
Cmet0931
Cmet0932
Cmet0933
Cmet0934
C

In [17]:
def show_info(pt):
    # rtss = data_path / f"train/rtss/{pt}.npy"
    # if rtss.exists():
    #     tumor_info = np.load(rtss)
    #     #print(f"Tumor info:\n{tumor_info}")
    #     img = np.load(data_path / f"train/anonymized/{pt}.npy")
    # else:
    #     img = np.load(data_path / f"train/controls/{pt}.npy")
    
    img = np.load(pt)

    plt.figure()
    plt.imshow(img)

limit = 0
for pt in data_path.glob("train/*"):
    if limit == 0:
        break
    limit -= 1

    show_info(str(pt))